<a href="https://colab.research.google.com/github/giannicha/aiffel/blob/main/EXP_06_writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환 

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


STEP03. 데이터정제

In [ ]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence)== 0 : continue
    if sentence[-1]== ':': continue
    
    if idx >9 :break
    
    print(sentence)


Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [ ]:
import os, re 
import numpy as np
import tensorflow as tf

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #일단 모두 소문자로 변환하고 양쪽 공백을 지우기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 아래 특수문자 기호 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백이 많을 수 있는 부분에는 하나의 공백으로 통일
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # ""안에 들어가있는 기호들 외에 공백으로 바꾸기
    sentence = sentence.strip() #다시 양쪽 공백을 지우기
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작과 끝에 start와 end 를 추가
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [ ]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 저장
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]
        
         

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

STEP04_평가 데이터셋 분리

In [ ]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들기
    # 우리는 이미 문장을 정제했으니 filters가 필요없음
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾸기
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    
    tokenizer.fit_on_texts(corpus) #위에서 만든 문장을 토크나이저에 넣어 데이터를 구축 
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus) 
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞추기
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=maxlen)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...    0    0    0]
 ...
 [   2  130    5 ...    0    0    0]
 [ 287   79  162 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f7013050fd0>


In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx > 9: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [ ]:
#마지막 토큰을 잘르기 위에서 end라고 설정했지만 문장 길이 상 pad인 것이 많다.
src_input = tensor[:, :-1]
#앞에 start부분을 자르기 
tgt_input = tensor[:, 1:]
print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
    0    0    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0
    0    0    0    0    0]


step04_ 평가 데이터 셋 분리

In [ ]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)

In [ ]:
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (140599, 19)
Target Train:  (140599, 19)


step05_인공지능 만들기

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
#문장을 토큰으로 했을 때 19이므로 19로 설정
embedding_size = 19
hidden_size = 2048

#여기서 tokenizer.num_words + 1 를 했는데 그 이유는 문장에 없는 pad 가 넣어졌기 때문
#문장길이를 모두 통일 하기 위해 가장 긴문장 말고는 모든 토큰이 0으로 들어간 부분 때문
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

history = []
epochs = 10

optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


In [ ]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 19, 12001), dtype=float32, numpy=
array([[[-2.73037731e-05,  1.17473792e-05, -3.45171757e-05, ...,
          5.79083535e-05, -4.37425479e-06,  9.86227133e-06],
        [ 4.36573373e-06,  1.03899940e-04, -4.96865687e-05, ...,
          3.16602454e-05, -5.25521609e-05,  4.63086617e-05],
        [ 3.01694654e-05,  1.43008758e-04, -3.44213840e-05, ...,
         -2.23296629e-05, -6.49107969e-05,  8.48639247e-05],
        ...,
        [ 4.63791686e-04,  4.51017055e-04, -4.05240979e-04, ...,
          6.10057963e-04, -3.12272401e-04,  3.09568131e-04],
        [ 5.04185620e-04,  5.15167776e-04, -4.49760526e-04, ...,
          7.19565491e-04, -3.45233944e-04,  3.65017302e-04],
        [ 5.38956607e-04,  5.68831398e-04, -4.91960032e-04, ...,
          8.15629493e-04, -3.74704745e-04,  4.14770882e-04]],

       [[-2.73037731e-05,  1.17473792e-05, -3.45171757e-05, ...,
          5.79083535e-05, -4.37425479e-06,  9.86227133e-06],
        [-3.09688803e-05,  1.21031502e-04, -9

In [ ]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  228019    
_________________________________________________________________
lstm (LSTM)                  multiple                  16941056  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 75,321,748
Trainable params: 75,321,748
Non-trainable params: 0
_________________________________________________________________


- 모델적용

In [ ]:
history = model.fit(enc_train, 
          dec_train, 
          epochs=epochs,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
550/550 [==============================] - 390s 702ms/step - loss: 3.0848 - val_loss: 2.7216
Epoch 2/10
550/550 [==============================] - 390s 710ms/step - loss: 2.5785 - val_loss: 2.4813
Epoch 3/10
550/550 [==============================] - 391s 712ms/step - loss: 2.3752 - val_loss: 2.3450
Epoch 4/10
550/550 [==============================] - 392s 712ms/step - loss: 2.2226 - val_loss: 2.2498
Epoch 5/10
550/550 [==============================] - 392s 713ms/step - loss: 2.0758 - val_loss: 2.1750
Epoch 6/10
550/550 [==============================] - 392s 712ms/step - loss: 1.9367 - val_loss: 2.1174
Epoch 7/10
550/550 [==============================] - 392s 712ms/step - loss: 1.8031 - val_loss: 2.0678
Epoch 8/10
550/550 [==============================] - 392s 712ms/step - loss: 1.6726 - val_loss: 2.0270
Epoch 9/10
550/550 [==============================] - 392s 713ms/step - loss: 1.5483 - val_loss: 1.9973
Epoch 10/10
550/550 [==============================] - 392s 714m

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 된다. 
                # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 준다
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측하지 않았거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much , i love you so much , i love you so much <end> '